In [2]:
from datasets import load_dataset
from verbalist.generation.generation_utils import VerbalistConversation, generate

dataset = load_dataset("dim/mt_bench_ru")

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_ru-26bc0c5acce9c7ed/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 114.16it/s]


In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

weights_path = "./verbalist/model/models/verbalist_7b_v1/"
# tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/saiga_7b_v1"
tokenizer_path = "./verbalist/model/models/verbalist_7b_v1/"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
generation_config = GenerationConfig(
    bos_token_id=1,
    eos_token_id=2,
    pad_token_id=0,
    max_new_tokens=512,
    # no_repeat_ngram_size=15,
    repetition_penalty=1.1,
    # temperature=1.0,
    top_k=40,
    top_p=0.95,
)
# generation_config = GenerationConfig.from_pretrained(tokenizer_path)
# print(generation_config)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.27s/it]
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/kosenko/miniconda3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`. This was detected when initializing the generatio

In [6]:
generation_config.do_sample = False
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    """Заполни пропущенные слова в предложении.
Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство""",
    """Россия лигитимное государство""",
    """
Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
Перепиши данное предложение на английском
""",
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    """
Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
""",
    """
Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
Тема: Опиумные войны между Китаем и Великобританией
Класс: 9-10
""",
    """
Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
""",
    """
Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
""",
]
for inp in inputs:
    conversation = VerbalistConversation(
        system_prompt="Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. Ты всегда отвечаешь на русском языке."
    )
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    print("PROMPT", prompt)
    generation_config = GenerationConfig(
        bos_token_id=1,
        eos_token_id=2,
        pad_token_id=0,
        max_new_tokens=512,
        # no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        # temperature=1.0,
        top_k=40,
        top_p=0.95,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

PROMPT <s> system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. Ты всегда отвечаешь на русском языке.</s>
<s> user
Почему трава зеленая?</s>
<s> bot
Почему трава зеленая?
The color of grass is determined by the pigments in its leaves and stems. These pigments are called chlorophylls, which absorb sunlight to produce energy for photosynthesis. During the growing season, when plants need to make food, they contain high levels of chlorophyll, which gives them a green color. When the growing season ends and the plant begins to prepare for winter, it stops making food and starts storing energy, reducing the amount of chlorophyll in its leaves. This causes the leaves to turn yellow or brown and the grass to become dormant. The remaining chlorophyll continues to break down, resulting in a reddish-brown color known as "fall color." So, the reason why grass is green during the growing season is because it contains high levels of chlorophyll, whic

: 